In [1]:
from keras.utils import HDF5Matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.utils import to_categorical
import tensorflow.compat.v1 as tf
import keras
from keras.layers import Input, Dense, BatchNormalization, Dropout, Conv2D, MaxPool2D, Flatten, LeakyReLU, Concatenate, UpSampling2D
import numpy as np
import pandas as pd
import imageio
from IPython.display import Image

import src.pcam_loader as data

Using TensorFlow backend.


## Run tensorflow keras on multiple core

In [2]:
# config = tf.ConfigProto(device_count={"CPU": 8})
# tensorflow.compat.v1.keras.backend.set_session(tf.Session(config=config))
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1660 Ti, pci bus id: 0000:26:00.0, compute capability: 7.5



## Load pcam data

In [3]:
# Load data from /User/Username/.keras/datasets/pcam
dataset = data.load_data()
x_train, y_train, meta_train =  dataset[0]
x_valid, y_valid, meta_valid =  dataset[1]
x_test, y_test, meta_test =  dataset[2]

In [4]:
y_train = np.array(y_train[:])
y_train = y_train.reshape(-1, 1)
y_valid = np.array(y_valid[:])
y_valid = y_valid.reshape(-1, 1)
y_test = np.array(y_test[:])
y_test = y_test.reshape(-1, 1)

In [5]:
batch_size = 64
epochs = 10
num_classes = 2
shape = (96, 96, 3)

In [6]:
# On 5 epoch get 77% on test, on valid 81%
# On 1 epoch get 75% on test, on valid %
# On 10 epoch get % on test, on valid %

#Input of CNN
input_layer = Input(shape=shape)

# First layer encoder with 2 Conv and MaxPooling
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
conv1 = Dropout(0.2)(conv1)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPool2D((2, 2))(conv1)

# Second layer encoder with 2 Conv and MaxPooling
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Dropout(0.2)(conv2)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPool2D((2, 2))(conv2)

# Third layer decoder with 2 Conv and Dropout
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Dropout(0.2)(conv3)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

# Fourth layer decoder with 2 Conv and Dropout
up1 = Concatenate(axis=-1)([UpSampling2D((2, 2))(conv3), conv2])
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = Dropout(0.2)(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

# Fifth layer decoder with 2 Conv and Dropout
up2 = Concatenate(axis=-1)([UpSampling2D((2, 2))(conv4), conv1])
conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up2)
conv5 = Dropout(0.2)(conv5)
conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)

# Output of classification model
output = Conv2D( num_classes, (1, 1), padding='same')(conv5)
output = Flatten()(output)
output = Dense(num_classes, activation='softmax')(output)

model = Model(input=[input_layer], output=[output])


C:\Users\matho\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


# Compile model and build model

In [7]:

model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 96, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 96, 96, 32)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 96, 96, 32)   9248        dropout_1[0][0]                  
____________________________________________________________________________________________

### Train the model

In [8]:
model.fit(
    x_train,
    to_categorical(y_train),
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(x_valid, to_categorical(y_valid)),
    shuffle='batch',
    # Add multiprocessing and workers
    use_multiprocessing=True, 
    workers=8
)

Train on 262144 samples, validate on 32768 samples
Epoch 1/10
262144/262144 [==============================] - 880s 3ms/step - loss: 1.4111 - accuracy: 0.7290 - val_loss: 0.4869 - val_accuracy: 0.7508
Epoch 2/10
262144/262144 [==============================] - 864s 3ms/step - loss: 0.4587 - accuracy: 0.7880 - val_loss: 0.5560 - val_accuracy: 0.7398
Epoch 3/10
262144/262144 [==============================] - 863s 3ms/step - loss: 0.4481 - accuracy: 0.7968 - val_loss: 0.4444 - val_accuracy: 0.7927
Epoch 4/10
262144/262144 [==============================] - 856s 3ms/step - loss: 0.4143 - accuracy: 0.8151 - val_loss: 0.4373 - val_accuracy: 0.7981
Epoch 5/10
262144/262144 [==============================] - 855s 3ms/step - loss: 0.3995 - accuracy: 0.8245 - val_loss: 0.4590 - val_accuracy: 0.8006
Epoch 6/10
262144/262144 [==============================] - 855s 3ms/step - loss: 0.3946 - accuracy: 0.8267 - val_loss: 0.4654 - val_accuracy: 0.7837
Epoch 7/10
262144/262144 [=======================

In [9]:
model.save("model_autoencoder_10epoch.h5")

In [10]:
test_eval = model.evaluate(x_test, to_categorical(y_test), verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

Test loss: 0.446651772770565
Test accuracy: 0.794647216796875
